# 一个简单的利用redis来做任务去重以加快数据处理速度的框架.
* 使用场景
适合一个对象会经常因为子对象变更而产生复杂计算, 并且对于这个计算允许有一定时延的场景
    1. 12306网站, 用户下单后, 需要重新计算当次列车剩余的车票数量.
    2. 股票网站, 因为某个人的交易, 需要更新5分钟的交易量数据

In [1]:
import redis
client = redis.Redis(decode_responses=True)

In [2]:
# 一个缓存用来保存数据
CACHE_KEY = "testcase_result"
# 一个缓存用来对任务去重
TASK_KEY = "testcase_to_update"

In [3]:
client.hset(CACHE_KEY,
            "testcase_0",
            json.dumps({"passed": False}),
           )
client.zadd(TASK_KEY,
            {"testcase_0": int(time.time()) - 1704000000}   # 每次都把时间戳当作权重
           )
client.hset(CACHE_KEY, "testcase_1", 
            json.dumps({"passed": True}),
           )
client.zadd(TASK_KEY, 
            {"testcase_1": int(time.time()) - 1704000000}   # 每次都把时间戳当作权重
           )
client.hset(CACHE_KEY, "testcase_0", 
            json.dumps({"passed": True}),
           )
client.zadd(TASK_KEY, 
            {"testcase_0": int(time.time()) - 1704000000}   # 每次都把时间戳当作权重
           )

0

In [4]:
"""虽然任务有触发3次, 但是实际只执行2次"""
print(client.zcard(TASK_KEY))
for (testcase_code, score) in client.zpopmin(TASK_KEY, count=100):
    result = client.hget(CACHE_KEY, testcase_code)
    client.hdel(CACHE_KEY, testcase_code)
    print(f"{testcase_code}变成了 {json.loads(result)}")

2
testcase_0变成了 {'passed': True}
testcase_1变成了 {'passed': True}
